# Augmented answer generate

## Imports

In [1]:
from fastai import *
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import itertools
from sklearn.metrics import roc_auc_score
from skopt import gp_minimize
from fastai import *
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import pickle

## Load data

In [2]:
df_train = pd.read_csv('data/train.csv')
X_train = df_train.drop(['ID_code', 'target'], axis=1)
y_train = df_train.target

## Train super model

In [ ]:
def load_model(model_name):
    with open(model_name, 'rb') as handle:
        model = pickle.load(handle)
    return model    

def make_prediction(model, X):
    return model.predict(X)

def get_all_preds(X, model_names):
    all_preds = {}
    for model_name in model_names:
        i = 0
        models = load_model(model_name)
        for model in models:
            i = i + 1
            y_pred = make_prediction(model, X)
            key = str(model_name) + str(i)
            all_preds[key] = y_pred
            print(f"{key} Done with {len(y_pred)} shape")
    return pd.DataFrame(all_preds)      

In [ ]:
model_names = !ls *m
model_names = list(model_names)

In [ ]:
X_aug = get_all_preds(X_train, model_names)

In [ ]:
#train super model (linear regression) on X_aug and y_train now

In [ ]:
X_aug.to_csv('train_augmented.csv', index=False)

### Get preds on test set

In [3]:
df_test = pd.read_csv('data/test.csv')
test_labels = df_test.ID_code
X_test = df_test.drop('ID_code', axis=1)

In [ ]:
X_test_aug = get_all_preds(X_test, model_names)

In [ ]:
X_test_aug.to_csv('test_augmented.csv', index=False)

# Super-learner

### Simple linear regression

In [ ]:
!ls *.csv

In [ ]:
augmented_train_x = pd.read_csv('train_augmented.csv')
augmented_y = y_train.copy()

In [ ]:
from sklearn import linear_model

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(augmented_train_x, augmented_y)

In [ ]:
reg.predict(augmented_train_x)

In [ ]:
augmented_test_x = pd.read_csv('test_augmented.csv')
augmented_test_x = augmented_test_x.drop('Unnamed: 0', axis = 1)

In [ ]:
psss = reg.predict(augmented_test_x)

In [ ]:
answer_df = pd.DataFrame({
    'ID_code' : test_labels,
    'target' : psss
})
answer_df.to_csv('answer_4_lgbm_with_augmented_data_smote_and_shiz_and_super_learner.csv', index=False)

### Submit average

In [ ]:
augmented_test_x = pd.read_csv('test_augmented.csv')

In [ ]:
augmented_test_x = augmented_test_x.drop('Unnamed: 0', axis = 1)

In [ ]:
predss = np.average(augmented_test_x, axis=1)

In [ ]:
test_labels.shape

# Evaluate all models

In [ ]:
from sklearn.model_selection import train_test_split
!ls *csv

In [ ]:
all_augments = pd.read_csv('train_augmented.csv')
all_augments.head()

In [ ]:
for column in all_augments:
    if 'random_aug_lgbm' in column:
        y_pred = all_augments[column]
        score = roc_auc_score(y_train, y_pred)
        print(f"Score for column {column} is {score}")

In [ ]:
# Let's try making a prediction only for random wale models

In [ ]:
test_augments = pd.read_csv('test_augmented.csv')
test_augments.head()

In [ ]:
test_augments.columns

In [ ]:
preds = {}
for column in test_augments:
    if 'random_aug_lgbm' in column:
        preds[column] = test_augments[column]

In [ ]:
random_preds = np.average(pd.DataFrame(preds), axis = 1)

In [ ]:
answer_df = pd.DataFrame({
    'ID_code' : test_labels,
    'target' : random_preds
})
answer_df.to_csv('answer_5_lgbm_with_only_random_augmented_data.csv', index=False)

In [ ]:
# Try averaging with old non augmented model

In [5]:
old_answer = pd.read_csv('answer_2_lgbm_with_CV10.csv')

In [ ]:
old_answer = old_answer.target

In [ ]:
old_answer.head()

In [ ]:
new_preds = pd.DataFrame(preds)
print(new_preds.shape)
new_preds.head()

In [ ]:
new_preds['old'] = 10 * old_answer
new_preds.head()

In [ ]:
a = new_preds.sum(axis = 1)/15

In [ ]:
answer_df = pd.DataFrame({
    'ID_code' : test_labels,
    'target' : a
})
answer_df.to_csv('answer_6_lgbm_with_best_and_random_averaged.csv', index=False)

### OR Gate between the two models

In [4]:
train_augments = pd.read_csv('train_augmented.csv')
test_augments = pd.read_csv('test_augmented.csv')

In [ ]:
train_augments.head()

In [ ]:
test_augments.head()

In [ ]:
y_train.head()

In [ ]:
roc_auc_score(y_train, train_augments['random_aug_lgbm_models_cv10.m3'])

In [7]:
old_answer = pd.read_csv('answer_2_lgbm_with_CV10.csv')
print(len(old_answer[old_answer.target>0.5]))
old_answer[old_answer.target>0.5].head()

6189


,ID_code,target
84,test_84,0.586097
121,test_121,0.547534
217,test_217,0.727779
431,test_431,0.745835
452,test_452,0.606438


In [8]:
preds = {}
for column in test_augments:
    if 'random_aug_lgbm' in column:
        preds[column] = test_augments[column]
random_preds = pd.DataFrame(preds)        

In [9]:
r_preds = np.average(random_preds, axis = 1)

In [10]:
r_preds = pd.Series(r_preds)
print(len(r_preds[r_preds>0.5]))
r_preds[r_preds>0.5].head()

22891


1     0.537717
3     0.544490
10    0.530922
16    0.621881
20    0.738424
dtype: float64

#### Simple OR

In [11]:
oldAnswer = old_answer.copy()
for index in range(len(oldAnswer)):
    cur = r_preds[index]
    if cur > 0.5:
        oldAnswer.target[index] = cur

/home/timetraveller/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [16]:
oldAnswer.to_csv('answer_7_lgbm_ORbwRandomAndBest.csv', index=False)

#### Averaged OR

In [17]:
avoldAnswer = old_answer.copy()
for index in range(len(avoldAnswer)):
    cur = r_preds[index]
    if cur > 0.5:
        avoldAnswer.target[index] = (avoldAnswer.target[index] + cur) / 2

/home/timetraveller/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [18]:
oldAnswer.to_csv('answer_8_lgbm_AveragedORbwRandomAndBest.csv', index=False)